# Продвинутый pandas 2

Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца 
timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating = ratings.groupby('userId').agg({'timestamp':['max', 'min', 'count']}).reset_index()
rating_new = rating[rating['timestamp']['count']>=100]
round((rating_new['timestamp']['max']-rating_new['timestamp']['min']).mean()/3600/24, 1)

511.5

In [4]:
# Или можно решить через merge

In [5]:
rt = ratings.groupby('userId').agg({'rating': 'count'}).reset_index()
rt_correct = rt[rt.rating>=100]
rt_correct.shape

(248, 2)

In [6]:
joined = ratings.merge(rt_correct, how='inner', on ='userId')
joined.head(5)

,userId,movieId,rating_x,timestamp,rating_y
0,1,1,4.0,964982703,232
1,1,3,4.0,964981247,232
2,1,6,4.0,964982224,232
3,1,47,5.0,964983815,232
4,1,50,5.0,964982931,232


In [7]:
joined_new = joined.groupby('userId').agg({'timestamp': ['max', 'min']})
joined_new.shape

(248, 2)

In [8]:
joined_new['diff'] = joined_new['timestamp']['max']-joined_new['timestamp']['min']
round(joined_new['diff'].mean()/86400, 1)

511.5

Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd = rzd.groupby('client_id').agg({'rzd_revenue': 'sum'}).reset_index()
rzd 

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto = auto.groupby('client_id').agg({'auto_revenue': 'sum'}).reset_index()
auto 

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air = air.groupby('client_id').agg({'air_revenue': 'sum'}).reset_index()
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
client_full = client_base.merge(rzd, how='left').merge(auto, how='left').merge(air, how='left')
client_full_with_address = client_full.fillna(0)

In [14]:
client_full_without_address = client_full_with_address.drop(['address'], axis='columns')
client_full_without_address

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [15]:
client_full_with_address

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т. е. для каждого user_id известна последовательность координат (широта/долгота), 
когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу 
визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки 
скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. 
Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе 
мало что дают.

В этой задаче так много вопросов и так мало ответов :)

Фактом визита считать нахождение пользователя в определенном радиусе от нашего магазина, 
т.е. считаем ближайшее расстояние до нашего магазина. 
Факт покупки, если пользователь находится по координатам рядом/в нашем магазине и в это же время 
происходит покупка.
Соответственно, считать эти расстояния по каждому пользователю и для каждого магазина(если у нас их несколько).
По координатам можно определить адреса, где пользователь чаще всего бывает, тем самым узнаем о его 
предпочтениях, отношении к конкурентам и тд. 
Если пользователь находится около нашего магазина можем оповещать его об акциях в конкретном магазине. 